In [1]:
import notebook_setup
from notebook_setup import MODELS_DIR

import numpy as np
import pandas as pd
import torch 
import timeit

from gpmdm import GPMDM, GPMDM_PF
from dataset_utils.mocap_labels import WALK_TRIALS_TEST, RUN_TRIALS_TEST
import dataset_utils.select_joints as select_joints

In [2]:
model_path = MODELS_DIR / 'gpmdm' / 'gpmdm_4d_30fps.pth'
gpdm = GPMDM.load(model_path)


Model and hyperparameters correctly loaded


In [3]:
DOWNSAMPLE = 4

TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")

Number of DOFs: 35


In [4]:
# Markov Transition Matrix between classes
markov_transition_matrix = torch.tensor(
    [[0.9, 0.1], 
     [0.1, 0.9]])

# Build a gpdm_pf model
gpdm_pf = GPMDM_PF(gpdm, 
                   markov_switching_model=markov_transition_matrix,
                   num_particles=100)
   

In [5]:
predicted_walk_actual_walk = 0
predicted_walk_actual_run = 0
predicted_run_actual_walk = 0
predicted_run_actual_run = 0
per_frame_times = []

trial_predicted_walk_actual_walk = 0
trial_predicted_walk_actual_run = 0
trial_predicted_run_actual_walk = 0
trial_predicted_run_actual_run = 0

for i, trial in enumerate(TEST_TRIALS):
    label = TRUTH_LABELS[i]

    # Get the data in an array format
    data_arr = trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::DOWNSAMPLE, :]

    gpdm_pf.reset()
    correct = 0
    total = 0

    # Run the filter bank on the trial
    for data in data_arr:

        start = timeit.default_timer()
        gpdm_pf.update(data)
        predicted_label = gpdm_pf.get_most_likely_class()
        latest_lls = gpdm_pf.class_probabilities()
        stop = timeit.default_timer()
        per_frame_times.append(stop - start)

        walk_ll = latest_lls[0]
        run_ll = latest_lls[1]

        if predicted_label == 0 and label == 0:
            predicted_walk_actual_walk += 1
        elif predicted_label == 0 and label == 1:   
            predicted_walk_actual_run += 1
        elif predicted_label == 1 and label == 0:
            predicted_run_actual_walk += 1
        elif predicted_label == 1 and label == 1:
            predicted_run_actual_run += 1

        if predicted_label == label:
            correct += 1

        total += 1

    accuracy = correct / total

    if accuracy > 0.5:
        if label == 0:
            trial_predicted_walk_actual_walk += 1
        else:
            trial_predicted_run_actual_run += 1

    else:
        if label == 0:
            trial_predicted_run_actual_walk += 1
        else:
            trial_predicted_walk_actual_run += 1


    print(f"Trial {i}, Accuracy: {accuracy}")
    print(f"Trial {i} Final, True label: {label}, Predicted label: {predicted_label}, Walk LL: {walk_ll}, Run LL: {run_ll}")

Trial 0, Accuracy: 0.972972972972973
Trial 0 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 1, Accuracy: 1.0
Trial 1 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 2, Accuracy: 0.9444444444444444
Trial 2 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 3, Accuracy: 0.9444444444444444
Trial 3 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 4, Accuracy: 0.9714285714285714
Trial 4 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 5, Accuracy: 1.0
Trial 5 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 6, Accuracy: 0.9761904761904762
Trial 6 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 7, Accuracy: 1.0
Trial 7 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 8, Accuracy: 1.0
Trial 8 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 9, Accuracy: 0.97826086956521

In [6]:
print('Results for Trials')

# Print the confusion matrix for trials
confusion_matrix = pd.DataFrame({
    'Actual Walk': [trial_predicted_walk_actual_walk, trial_predicted_run_actual_walk],
    'Actual Run': [trial_predicted_walk_actual_run, trial_predicted_run_actual_run]
}, index=['Predicted Walk', 'Predicted Run'])

# add the total column and row
confusion_matrix['Total'] = confusion_matrix.sum(axis=1)
confusion_matrix.loc['Total'] = confusion_matrix.sum(axis=0)

print(confusion_matrix)

# Calculate the accuracy, precision, recall, and F1 score for trials
accuracy = (trial_predicted_walk_actual_walk + trial_predicted_run_actual_run) / (trial_predicted_walk_actual_walk + trial_predicted_walk_actual_run + trial_predicted_run_actual_walk + trial_predicted_run_actual_run)
precision = trial_predicted_walk_actual_walk / (trial_predicted_walk_actual_walk + trial_predicted_run_actual_walk)
recall = trial_predicted_walk_actual_walk / (trial_predicted_walk_actual_walk + trial_predicted_walk_actual_run)
f1 = 2 * precision * recall / (precision + recall)


print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Print the average time per frame
print(f"Average time per frame: {np.mean(per_frame_times)}")
print(f"Average FPS: {1 / np.mean(per_frame_times)}")

print('\nResults for Frames')

# Print the confusion matrix for frames
confusion_matrix = pd.DataFrame({
    'Actual Walk': [predicted_walk_actual_walk, predicted_run_actual_walk],
    'Actual Run': [predicted_walk_actual_run, predicted_run_actual_run]
}, index=['Predicted Walk', 'Predicted Run'])

# add the total column and row
confusion_matrix['Total'] = confusion_matrix.sum(axis=1)
confusion_matrix.loc['Total'] = confusion_matrix.sum(axis=0)

print(confusion_matrix)

# Calculate the accuracy, precision, recall, and F1 score for frames
accuracy = (predicted_walk_actual_walk + predicted_run_actual_run) / (predicted_walk_actual_walk + predicted_walk_actual_run + predicted_run_actual_walk + predicted_run_actual_run)
precision = predicted_walk_actual_walk / (predicted_walk_actual_walk + predicted_run_actual_walk)
recall = predicted_walk_actual_walk / (predicted_walk_actual_walk + predicted_walk_actual_run)
f1 = 2 * precision * recall / (precision + recall)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Results for Trials
                Actual Walk  Actual Run  Total
Predicted Walk           20           0     20
Predicted Run             1          18     19
Total                    21          18     39
Accuracy: 0.9743589743589743
Precision: 0.9523809523809523
Recall: 1.0
F1 Score: 0.975609756097561
Average time per frame: 0.07824731495460323
Average FPS: 12.779991244174582

Results for Frames
                Actual Walk  Actual Run  Total
Predicted Walk         1704           8   1712
Predicted Run           199         717    916
Total                  1903         725   2628
Accuracy: 0.9212328767123288
Precision: 0.8954282711508145
Recall: 0.9953271028037384
F1 Score: 0.9427385892116182
